In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModel
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset
import torch

In [3]:
tokenizer = AutoTokenizer.from_pretrained('deepset/sentence_bert')
model = AutoModel.from_pretrained('deepset/sentence_bert')

In [4]:
df = pd.read_csv("../data/filtered_amazon_co-ecommerce_sample.csv")

In [6]:
descriptions = list(df['description'])
print(len(descriptions))

8514


In [56]:
inputs = tokenizer.batch_encode_plus(descriptions,
                                     return_tensors='pt',
                                     padding='max_length',
                                     max_length=512,
                                     truncation=True)


In [57]:
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], inputs['token_type_ids'])

In [58]:
dataloader = DataLoader(dataset, batch_size=16)


In [66]:
def to_list(tensor):
    return tensor.detach().cpu().tolist()

In [63]:
all_sentences = []

for batch in tqdm(dataloader, desc="Computing sentence representations"):
    inputs = {
              "input_ids": batch[0],
              "attention_mask": batch[1],
              "token_type_ids": batch[2],
          }
    
    outputs = model(**inputs)[0]
    sentence_reps = to_list(outputs.mean(dim=1))
    all_sentences.append(sentence_reps)  
    
    break

Computing sentence representations:   0%|          | 0/533 [00:19<?, ?it/s]

[[-0.15184307098388672, 0.7549017667770386, 0.060852330178022385, 0.04276290535926819, -0.23419103026390076, 0.41326114535331726, -0.1659739464521408, 0.15801531076431274, -0.25383755564689636, 0.20442424714565277, -0.5000050663948059, 0.09854844212532043, -0.3672836422920227, 0.11794833838939667, 0.0626024529337883, -0.09168116003274918, -0.24958190321922302, -0.3959147334098816, 0.13880395889282227, 0.02817206084728241, -0.32433071732521057, -0.9157824516296387, 0.06564252823591232, -0.5794345140457153, -0.13022799789905548, 0.5227569937705994, 0.007175183389335871, -0.3973822295665741, -0.11186329275369644, -0.28327780961990356, -0.0725930780172348, -0.43002334237098694, 0.5758967399597168, -0.5431287288665771, -0.4906271696090698, 0.3346789479255676, -0.682158887386322, -0.31482088565826416, 0.01668461225926876, 0.19553479552268982, 0.23940618336200714, 0.09271647781133652, -0.4112212359905243, -0.06409133970737457, -0.9973798990249634, 0.7797624468803406, -0.7892370820045471, 0.04

In [65]:
len(sentence_reps[0])

768

In [54]:
output = model(**inputs)[0]
sentence_rep = output.mean(dim=1)


RuntimeError: The size of tensor a (768) must match the size of tensor b (512) at non-singleton dimension 1